In [ ]:
# SSH tunnel setup (not functional right now)

# SSH tunnel setup (Zheng Li)
from sshtunnel import SSHTunnelForwarder
import socket
from time import sleep

icebloc_ip = '192.168.1.222'
computer_public_ip = '18.25.25.243'  #check the computer public ip in 448

computer_interface_ip = '192.168.1.100' #ip seen by the icebloc
remote_interface_port= 39933  #talk between icebloc and the computer in 448
local_port=9001


server = SSHTunnelForwarder(
    computer_public_ip,
    ssh_username="User",
    ssh_password="POE6730gp",
    remote_bind_address=(icebloc_ip, 39933),
    local_bind_address=('127.0.0.1',local_port)
)

server.start()

print(server.local_bind_port)  # show assigned local port

# Setup Instruments

In [3]:
%reset


import sys
import drivers.redpitaya_scpi as scpi
import numpy as np
import matplotlib.pyplot as plt
import time 
from time import sleep
import socket
from datetime import datetime
import csv

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


ModuleNotFoundError: No module named 'drivers'

# laser set up and test

In [ ]:
from drivers.m2_solstis_v3 import M2_Solstis
from drivers.M2_tune_v4 import M2_tune
laser = M2_Solstis()

# test tuning
timestamp_str = datetime.strftime(datetime.now(),'%Y_%m_%d_%H_%M_%S')
success,mwl = M2_tune(laser,786.8,timestamp_str) 
print(success)
print(mwl)

# Optical switch connect and test

In [ ]:
# Test DICON optical switch connection
from instrumental.drivers.opticalswitch.diconopticalswitch import DiConOpticalSwitch
switch = DiConOpticalSwitch(port='COM3')
switch.identify()

In [ ]:
switch.set_channel(2)

In [ ]:
switch.get_channel()

# Power meter connection

### Troubleshooting Powermeter connection
1. if measPower is not working and throwing b'The specified mode is invalid.' error

Close tlPM and turn on Thorlabs Optical Power Meter Application

2. if measPower is not working and throwing  b'A USB control transfer stalled'

In [ ]:
# Open connection to Thorlabs PM101A powermeter
from datetime import datetime
from ctypes import cdll,c_long, c_ulong, c_uint32,byref,create_string_buffer,c_bool,c_char_p,c_int,c_int16,c_double, sizeof, c_voidp
from drivers.TLPM import TLPM
import time

try: 
    tlPM = TLPM()
    deviceCount = c_uint32()
    tlPM.findRsrc(byref(deviceCount))

    print("devices found: " + str(deviceCount.value))

    resourceName = create_string_buffer(1024)

    for i in range(0, deviceCount.value):
        tlPM.getRsrcName(c_int(i), resourceName)
        print(c_char_p(resourceName.raw).value)
        break

    tlPM.open(resourceName, c_bool(True), c_bool(True))

    message = create_string_buffer(1024)
    tlPM.getCalibrationMsg(message)
    print(c_char_p(message.raw).value)
    
except:
    print('Could not open Thorlabs Power meter : {}'.format(sys.exc_info()))
else:    
    print('Thorlabs Powermeter connected')    
    print('Setting power auto range on')
    tlPM.setPowerAutoRange(c_int16(1))
    print('Set Analog Output slope to 500V/W')
    tlPM.setAnalogOutputSlope(c_double(500))
    print('Setting input filter state to off for higher BW')
    tlPM.setInputFilterState(c_int16(0))
    #print('Setting power range to 5.5 mW')
    #tlPM.setPowerRange(c_double(5.5e-3))


# Set up Red pitaya signal and power monitoring channels and appropriate switch channels

In [ ]:
# Setup red pitaya 2 channels =: channel 0 is input and channel 1 is power monitoring from VOA
#integration_time = 2 # sec

# Address to Multimode and thorlabs MM tap monitoring Red pitaya misnomers below copied from Sweep_V1.6n

sm_voa_rp = 'rp-f08473.local' # channel 1 is SM probe (LV) and channel 2 is VOA (HV)
power_rp = 'rp-f08473.local' # channel 2 monitors power meter analog output

# Dictionary defining Redpitaya addresses to Dicon Switch channels, Data is always input into Channel 1 of RedPitayas
fiber_channel = {
    2: 'rp-f08473.local'
    
    #1: 'rp-f084D3.local', 
    #2: 'rp-f08358.local'  
}

# IP and SCPI objects for RedPitayas
ip_s = {}
rp_s = {}

# Get IP and connect to scpi server
try:
    ip_s.update({sm_voa_rp : socket.gethostbyname(sm_voa_rp)})        
    rp_s.update({sm_voa_rp : scpi.scpi(ip_s[sm_voa_rp])})
except:
    print('{} is unavailable: {}'.format(sm_voa_rp, sys.exc_info()))
else:
    print('SM VOA RedPitaya at {} has IP {} is connected'.format(sm_voa_rp, ip_s[sm_voa_rp]))

    
# 
for (channel,hostname) in fiber_channel.items():
    # Get IP and connect to scpi server
    try:
        ip_s.update({hostname : socket.gethostbyname(hostname)})        
        rp_s.update({hostname : scpi.scpi(ip_s[hostname])})
    except:
        print('{} is unavailable: {}'.format(hostname, sys.exc_info()))
    else:
        print('MM channel {} RedPitaya at {} has IP {} is connected'.format(channel, hostname, ip_s[hostname]))
   
       

In [ ]:
# Acquisition parameters setup

In [ ]:
# Setup acquisition parameters for Redpitayas
# sampling_rate =1.2e6 # 1.90734863e+03 # 1.56250000e+07 # 1.25000000e+08 # 12e6
sampling_rate = 1000

# Modulation parameters.
wave_form = 'square'
carrier_freq = 10.0 # +0.11 # Hz
ampl = 0.5
offset = 0.5
# RedPitaya internal parameters
max_sampling_rate = 125e6 #sample/s
total_samples = 16384 #max buffer size?
if sampling_rate > max_sampling_rate:
    print('Maximum sampling rate is 125 MHz')
    sampling_rate = max_sampling_rate
decimation_options = np.array([65536, 8192, 1024, 64, 8, 1])
sampling_rates = max_sampling_rate/decimation_options

decimation = decimation_options[np.argmax(sampling_rate <= sampling_rates)]

sampling_freq = np.floor(max_sampling_rate/decimation)

print('decimation: {}, sampling freq = {}'.format(decimation, sampling_freq))

# This is Redpitaya 1 channel settings
if sm_voa_rp in rp_s:
    print('Setting signal acquisition parameters for {}'.format(sm_voa_rp))
    
    rp_s[sm_voa_rp].tx_txt('ACQ:RST')
    rp_s[sm_voa_rp].tx_txt('ACQ:AVG OFF') # turn off averaging        
    rp_s[sm_voa_rp].tx_txt('ACQ:DEC {}'.format(decimation)) # set decimation 
    #     rp.tx_txt('ACQ:SOUR1:COUP DC') # enables DC coupling on channel 1
    rp_s[sm_voa_rp].tx_txt('ACQ:SOUR1:GAIN LV') # set low voltage gain settings
    rp_s[sm_voa_rp].tx_txt('ACQ:SOUR2:GAIN HV') # set high voltage gain settings
    rp_s[sm_voa_rp].tx_txt('ACQ:TRIG NOW') # set to trigger immediately
else:
    print('{} is unavailable'.format(sm_voa_rp))

    # This is Redpitaya 2,3 channel settings
for (channel,hostname) in fiber_channel.items():
    # Get IP and connect to scpi server
    if hostname in rp_s:   
        print('Setting signal acquisition parameters for {} channel {}'.format(hostname, channel))
        rp_s[hostname].tx_txt('ACQ:RST')
        rp_s[hostname].tx_txt('ACQ:AVG OFF') # turn off averaging        
        rp_s[hostname].tx_txt('ACQ:DEC {}'.format(decimation)) # set decimation 
        #     rp.tx_txt('ACQ:SOUR1:COUP DC') # enables DC coupling on channel 1
        rp_s[hostname].tx_txt('ACQ:SOUR1:GAIN LV') # set low voltage gain settings
        rp_s[hostname].tx_txt('ACQ:SOUR2:GAIN HV') # set high voltage gain settings        
        rp_s[hostname].tx_txt('ACQ:TRIG NOW') # set to trigger immediately
    else:
        print('{} is unavailable'.format(hostname))
        

# Wavelength sweep and spectrum acquisition

In [ ]:
import matplotlib.pyplot as plt
# %matlplotlib notebook

# Alternative way of scanning wavelength range (going by wavelength, skipping if failed and returning later)

timestamp_str = datetime.strftime(datetime.now(),'%Y_%m_%d_%H_%M_%S')

fpath_log_csv = './data/'+timestamp_str+'_log.csv'

run_log = []

mm_sample = {}
sm_sample = input('What is the sample you are looking at SM channel? ')
for (channel, hostname) in fiber_channel.items():
    mm_sample[channel] = input('What is the sample you are looking at MM channel {}? '.format(channel))

run_log.append('sample, {} \n'.format(sm_sample))
run_log.append('sampling_freq, {} \n'.format(sampling_freq))

np.savetxt(fpath_log_csv, run_log, delimiter=",",fmt="%s",newline = "\n")

t0 = time.time()

start_wavelength=float(input('Start wavelength: '))
end_wavelength=float(input('End wavelength: '))
step_size=float(input('Step size: '))

run_log.append('start_wavelength,  {} \n '.format(start_wavelength))
run_log.append('end_wavelength, {} \n '.format(end_wavelength))
run_log.append('step_size, {} \n '.format(step_size))

# target_wavelengths = np.arange(start_wavelength, end_wavelength+step_size, step_size) # the array of wavlengths we are aiming for = 2 # to allow settling
#target_wavelengths = np.arange(start_wavelength, end_wavelength+step_size*0.1, step_size) # the array of wavlengths we are aiming for = 2 # to allow settling
target_wavelengths = [814.75, 815, 811.0, 811.25, 811.5]

integration_time = float(input('Integration time in s: ')) # seconds on each wavelength 
N_repetitions = int(input('Spectra repetitions: ')) # number of repeating each data points (wavelengths acquisitions)

lockin=input('Lock in Y/N: ')
lock_in=lockin.upper()

run_log.append('integration_time, {} \n'.format(integration_time))
run_log.append('N_repetitions, {} \n '.format(N_repetitions))
run_log.append('lock_in, {} \n'.format(lock_in))

np.savetxt(fpath_log_csv, run_log, delimiter=",", fmt="%s", newline="\n")

# initializing output data structures
voa_output_dic = {}  # time domain
sm_output_dic = {}
mm_output_dic = {}
power_output_dic = {} # time domain power monitoring

for (channel, hostname) in fiber_channel.items():
    mm_output_dic[channel] = []
    sm_output_dic[channel] = []
    voa_output_dic[channel] = []
    power_output_dic[channel] = []

# Setup power meter wavelength settings
avgTime = c_double()
# print( tlPM.getAvgTime(c_int16(2), byref(avgTime)) ) # previously set average time 
tlPM.setWavelength(c_double(785.0))
power = c_double()

failed_wavelength = target_wavelengths # wavelengths we will return to if their tuning fails

# Variables for plot
fig = None
plot = False

while len(failed_wavelength)!=0:

    target_wavelengths = failed_wavelength # setting targets as failed from previous
    
    print('failed wavelength = {}'.format(failed_wavelength))
    
    failed_wavelength = [] # emptying out the previous failed
    
    for wi in range(len(target_wavelengths)): 

        tune_success, measured_wavelength = M2_tune(laser,float(target_wavelengths[wi]),timestamp_str) # if 0 fail, move to next wavelength, else collect data 

        if tune_success == 0:

           failed_wavelength.append(target_wavelengths[wi])

        elif tune_success == 1:
            for (channel, hostname) in fiber_channel.items():
                switch.set_channel(channel) #remember to chnage back to (channel)

                for n in range (1,N_repetitions+1):

                    print('Acquisition {} of {}'.format(n,N_repetitions))

                    measured_wavelength = laser.poll_wavelength() # updated laser wavelength poll each acquisition

                    remain_time = integration_time # clearing buffers for new data collect

                    buff_sm = []
                    buff_mm = []
                    buff_voa = []    
                    buff_power = []                   
                   

                    while remain_time > 0.0:

                        if remain_time > total_samples/sampling_freq: # too many samples for one buffer
                            num_of_samples = total_samples                            
                        else:
                            num_of_samples = np.ceil(remain_time*sampling_freq)

                        print(' -> -> Acquire {} samples'.format(num_of_samples))
                        # set power meter averaging time
#                         tlPM.setAvgTime(c_double(num_of_samples/sampling_freq))                            

                        # Send Acquisition start/stop commands to all RedPitayas
                        rp_s[hostname].tx_txt('ACQ:START')  # acqiring signal
                        if hostname!=power_rp:
                            rp_s[power_rp].tx_txt('ACQ:START')  # acquire power meter monitoring signal
                        rp_s[sm_voa_rp].tx_txt('ACQ:START')  # acquire voa power monitoring and SM probe signal

                        sleep(num_of_samples/sampling_freq)

                        rp_s[hostname].tx_txt('ACQ:STOP')
                        if hostname!=power_rp:
                            rp_s[power_rp].tx_txt('ACQ:STOP')  
                        rp_s[sm_voa_rp].tx_txt('ACQ:STOP')

                        # Download SM and VOA power monitor data
                        #rp_s[sm_voa_rp].tx_txt('ACQ:SOUR1:DATA:LAT:N? {}'.format(num_of_samples))
                        rp_s[sm_voa_rp].tx_txt('ACQ:SOUR2:DATA:LAT:N? {}'.format(num_of_samples)) # HV
                        buff_string = rp_s[sm_voa_rp].rx_txt()
                        buff_sm = buff_sm + list(map(float, buff_string.strip('ERR!{}\n\r').split(',')))
                        
                        #rp_s[sm_voa_rp].tx_txt('ACQ:SOUR1:DATA:LAT:N? {}'.format(num_of_samples))
                        rp_s[sm_voa_rp].tx_txt('ACQ:SOUR1:DATA:LAT:N? {}'.format(num_of_samples)) # HV
                        buff_string = rp_s[sm_voa_rp].rx_txt()
                        buff_voa = buff_voa + list(map(float, buff_string.strip('ERR!{}\n\r').split(',')))
                        
                        # Download MM channel data
#                         rp_s[hostname].tx_txt('ACQ:SOUR1:DATA:LAT:N? {}'.format(num_of_samples))  # LV channel
                        rp_s[hostname].tx_txt('ACQ:SOUR2:DATA:LAT:N? {}'.format(num_of_samples)) # HV 
                        buff_string = rp_s[hostname].rx_txt()
                        buff_mm = buff_mm + list(map(float, buff_string.strip('ERR!{}\n\r').split(',')))      
                        
                        # Download Powermeter analog out data from channel 2 of powermeter redpitaya
#                         print( tlPM.measPower(byref(power)) )
#                         buff_power = buff_power + [float(power.value)]* len(buff_voa)
#                         rp_s[power_rp].tx_txt('ACQ:SOUR2:DATA:LAT:N? {}'.format(num_of_samples))  # HV
                        rp_s[power_rp].tx_txt('ACQ:SOUR1:DATA:LAT:N? {}'.format(num_of_samples)) # LV
                        buff_string = rp_s[power_rp].rx_txt()
                        buff_power = buff_power + list(map(float, buff_string.strip('ERR!{}\n\r').split(',')))      

#                         buff_s_string = rp_s[hostname].rx_txt()
#                         buff_v_string = rp_v[hostname].rx_txt()

#                         buff_s_string = buff_s_string.strip('ERR!{}\n\r').split(',')
#                         buff_v_string = buff_v_string.strip('ERR!{}\n\r').split(',')

#                         buff_s = buff_s + list(map(float, buff_s_string)) # adding buffers if needed
#                         buff_v = buff_v + list(map(float, buff_v_string))

                        
        

                        remain_time = remain_time - num_of_samples/sampling_freq

                        if remain_time>0:

                            print('Remaining time {}'.format(remain_time))


                        # fft and demodulate data of single buffer  - NOT COMPATIBLE WITH MULTI-BUFFER?

                        timevec = np.arange(0, len(buff_sm), 1)/sampling_freq
                        #freq = np.fft.fftfreq(len(buff_s), d=1/sampling_freq)


                        sm_output_dic[channel].append([measured_wavelength] + list(buff_sm)) # adding thw wavelength to the list
                        mm_output_dic[channel].append([measured_wavelength] + list(buff_mm)) # adding thw wavelength to the list
                        voa_output_dic[channel].append([measured_wavelength] + list(buff_voa))
                        power_output_dic[channel].append([measured_wavelength] + list(buff_power))
                        time_array_save = np.concatenate((np.array([0]), timevec))

                
                
                # save data each round  so to not lose all of it if the process fails                                
                print('Writing output csv files for channel {}'.format(channel))

                fpath_sm_csv = './data/'+timestamp_str+'_SM_' + str(channel)+'.csv'
                header_sm = sm_sample + ' measured on SM probe when optical switch on channel {}'.format(channel)
                np.savetxt(fpath_sm_csv, 
                           np.vstack((time_array_save, np.array(sm_output_dic[channel]))).T,
                           delimiter=",",fmt='%s') #, header=header_sm)

                fpath_mm_csv = './data/'+timestamp_str+'_MM_' + str(channel)+'.csv'
                header_mm = mm_sample[channel] + ' measured on MM probe when optical switch on channel {}'.format(channel)
                np.savetxt(fpath_mm_csv, 
                           np.vstack((time_array_save, np.array(mm_output_dic[channel]))).T,
                           delimiter=",",fmt='%s') #, header=header_sm)

#                 fpath_power_csv = './data/'+timestamp_str+'_power_' + str(channel)+'.csv'
#                 np.savetxt(fpath_power_csv, 
#                            np.vstack((time_array_save, np.array(power_output))).T,
#                            delimiter=",",fmt='%s')

                fpath_voa_csv = './data/'+timestamp_str+'_voa_' + str(channel)+'.csv'
                header_voa = 'VOA output when optical switch on channel {}'.format(channel)
                np.savetxt(fpath_voa_csv, 
                           np.vstack((time_array_save, np.array(voa_output_dic[channel]))).T,
                           delimiter=",",fmt='%s')
                #fpath_demod_csv = './data/'+timestamp_str+'_demod.csv'

#                 output_raw_save = np.array(output_raw)
#                 power_raw_save = np.array(power_raw)

                #freq_array_save =  np.array([np.concatenate((np.array([0]),freq))])


                fpath_power_csv = './data/'+timestamp_str+'_power_' + str(channel)+'.csv'
                header_power = 'Power meter output when optical switch on channel {}'.format(channel)
                np.savetxt(fpath_power_csv, 
                           np.vstack((time_array_save, np.array(power_output_dic[channel]))).T,
                           delimiter=",",fmt='%s') #, header = header_power)

                #output_demod_t = np.vstack((freq_array_save, output_demod)).T
                #output_demod_t_real = np.real(output_demod_t)
                #np.savetxt(fpath_demod_csv, output_demod_t_real, delimiter=",",fmt='%s') #, header=['Frequency', 'Measured Wavelength'])
                
                # Return power meter to default avgTime
#                 tlPM.setAvgTime(avgTime)

                if plot:
                    if fig is not None:
                        fig.clear()
                    fig, ax = plt.subplots(nrows=4, ncols=1, sharex=True, sharey=True, figsize=(10, 10))
                    fig.text(0.5, 0.04, 'Time [sec]', ha='center')
                    fig.text(0.04, 0.5, 'Voltage [V]', va='center', rotation='vertical')

                    ax[0].set_title('SM probe')
                    ax[0].plot(time_array_save[1:], sm_output_dic[channel][-1][1:], label='SM probe')                

                    ax[1].set_title('MM probe Channel {}'.format(channel))
                    ax[1].plot(time_array_save[1:], mm_output_dic[channel][-1][1:], label='MM probe')                

                    ax[2].set_title('VOA')
                    ax[2].plot(time_array_save[1:], voa_output_dic[channel][-1][1:], label='VOA')                

                    ax[3].set_title('Power meter')
                    ax[3].plot(time_array_save[1:], power_output_dic[channel][-1][1:], label='Power meter')                
                    plt.show()
       
  

    print(failed_wavelength)

    
print('finished')

t1 = time.time()

elapsed_run_time = t1-t0

run_log.append('elapsed_run_time, {} \n'.format(elapsed_run_time))

np.savetxt(fpath_log_csv, run_log, delimiter=",",fmt="%s",newline = "\n")

print('Experiment ran {} seconds'.format(elapsed_run_time))
             


# Closing down

In [ ]:
import sys
# closing everything
# solstis

try: 
    laser.close()
except:
    print('Could not close laser : {}'.format(sys.exc_info()))
else:
    print('M2 Solstis connection closed')
        
        
# Redpitayas
try:        
    for (hostname,rp_socket) in rp_s.items():           
        rp_socket.close()
        print('{} with IP {} successfully disconnected'.format(hostname, ip_s[hostname]))
except:
    print('Could not close redpitayas : {}'.format( sys.exc_info())) # hostname, sys.exc_info()))
    
    
        
# Dicon Switch
try: 
    switch.close()
except:
    print('Could not close Dicon Optical Switch : {}'.format(sys.exc_info()))
else:
    print('Dicon Optical Switch connection closed')
    
    
# Thorlabs PM101A Powermeter
try: 
    tlPM.close()
except:
    print('Could not close Thorlabs Power meter : {}'.format(sys.exc_info()))
else:
    print('Thorlabs Powermeter connection closed')
    